In [ ]:
%load_ext autoreload
%autoreload 2

# RHEED spots calculated using Ewald construction
This notebook shows how to use xrheed ewald module to superimpose the calculated spot positions on the RHEED image.

In [ ]:
import xrheed
from xrheed.io import load_data

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from pathlib import Path

## Prepare the RHEED image data
As an example we use the image recorded from the clean Si(111) surface with (7x7) superstructure. 

The electron beam was aligned along the $[11 \bar 2]$ direction. This means that horizontal direction on the RHEED image could be considered as $[1 \bar 1 0]$.

In [ ]:
image_dir = Path("example_data")
image_path = image_dir / "Si_111_7x7_112_phi_00.raw"

rheed_image = load_data(image_path, plugin="dsnp_arpes_raw")

# Rotate the image
rheed_image.ri.rotate(-0.4)

# Apply automatic center search again after rotation
rheed_image.ri.apply_image_center(auto_center=True)

rheed_image.ri.screen_roi_width = 60
rheed_image.ri.screen_roi_height = 80

# Use automatic levels adjustment
rheed_image.ri.plot_image(auto_levels=0.5)

plt.show()

## Prepare the 2D lattice object
First we can calculate the position of the (1x1) spots using the hexagonal 2D lattice with the 3.84 A lattice constant.

The lattice could be created by using `Lattice` class, where basic vectors of the real space are used in constructor.

There are also class other methods:
 - `from_bulk_cubic` - creates the lattice by providing the bulk lattice constant `a`, the `cubic_type`, and the `plane` (only low index planes are supported).
 - `from_surface_hex` - create the 2D hexagonal lattice by providing only a surface lattice constant 

 Below all options are presented.

In [ ]:
from xrheed.kinematics.lattice import Lattice

In [ ]:
# create lattice manually 
lattice = Lattice([3.84, 0], [3.84*0.5, 3.325])

# create lattice from a bulk cubic
lattice = Lattice.from_bulk_cubic(a=5.43, cubic_type="FCC", plane='111')

# create 2d hex lattice using dedicated method
lattice = Lattice.from_surface_hex(a=3.84)

In [ ]:
si_111_1x1 = Lattice.from_surface_hex(a=3.84)

si_111_1x1.plot_real(space_size=7.0)
plt.show()

Show reciprocal lattice (it's generated automatically)

In [ ]:
si_111_1x1.plot_reciprocal(space_size=3.0)
plt.show()

The hexagonal lattice is by default generated in a way that the `x` direction (that is close the electron beam direction) corresponds to the $[11\bar 2]$ direction.

If the sample orientation was different, the lattice could be rotated, but it's also possible to provide the `alpha` angle with the RHEED image. In the latter case the lattice will be rotated in Ewald class object.

When the lattice is rotated, both real and reciprocal representation that are updated automatically.

## Ewald construction

### RHEED experiment geometry
The sketch bellow shows the geometry used in `xRHEED` package.

![xRHEED geometry](../_static/xRHEED_geometry.svg)

To calculate the spot positions for a given lattice and experimental conditions, first we need to calculate the $\varphi$ and $\theta$ angles for each crossing point where Ewald sphere intersects with reciprocal lattice rods.

Then we use those angles in real space to calculate the $S_x$ and $S_y$ spot positions on the RHEED screen.

Those angles are defined in similar way to the spherical coordinate system but, here $\theta$ is measured in respect to the $x$ axes (instead of $z$ as typically used).


### Ewald sphere in k-space
The reciprocal lattice of a 2D real lattice has a form of regularly distributed rods that are normal to a crystal surface plane.

The diffraction spots are expected in the directions that are defined by the intersection points of the Ewald sphere and the rods, as shown on the sketch bellow.

![Ewald construction](../_static/Ewald_construction.svg)

While it is a common practice to show the sample, the screen and Ewald construction on the same sketch, it is misleading while this mixes real and reciprocal dimensions.

### Calculation algorithm for $\theta $ and $\varphi$
1. Calculate the Ewald sphere radius:
$$k_0 = \sqrt{E} * 0.5123$$
2. Generate the reciprocal rods in a (x,y) plane as $g_x$ , $g_y$.
3. Shift the reciprocal lattice along the $k_x$ direction to match the specular reflection with the crossing point of lattice rod and the sphere. 

The shift is calculated using following formula: 
$$\delta x = k_0 \cdot \cos(\beta)$$

Then we use $k_x$ and $k_y$ for shifted reciprocal lattice points.

4. Select only those reciprocal lattice rods that are inside the Ewald sphere (their distance from the center of the sphere is smaller that it's radius). Only those could possibly cross the Ewald sphere. 
5. Calculate the $r_k$ value for each point. 
$$r_k = \sqrt{k_0^2 - k_x^2}$$
6. Then, the angles are defined by following relations:
$$\theta = \arcsin{(r_k / k_0)}$$
$$\phi = \arccos{(k_x/ r_k)}$$

7. Having the $\theta$ and $\varphi$ values we can calculate the radius:
$$\rho = L \cdot \tan(\theta) $$
where $L$ is a screen-sample distance.
 
Finally (on the screen coordinates $s_x$, $s_y$): 
$$s_x = \rho \cdot \cos(\varphi)$$
$$s_y = \rho \cdot \sin(\varphi)$$

## Ewald class

Finally, having two necessary ingredients:
- loaded and aligned RHEED image,
- lattice object.

The spot position can be calculated using kinematic theory (Ewald construction) as shown below. 

In general the RHEED image is optional, however to calculate the spot position in an real experimental conditions 
the image is used to provide for example, screen-sample distance, screen scaling, incident theta angle.

Please note that, `Ewald` object creates a separate lattice for it's own purpose (this lattice could be easily scaled for precised matching).



In [ ]:
from xrheed.kinematics.ewald import Ewald

In [ ]:
ew_si_111 = Ewald(si_111_1x1, rheed_image)

ew_si_111

In [ ]:
ew_si_111.calculate_ewald()

In [ ]:
fig, ax = plt.subplots()

ew_si_111.plot(ax=ax, show_image=True, show_center_lines=False, auto_levels=1.0)
plt.show()

## Fine adjustment

As visible on the image above the incident angle is not set properly, also the image scaling sometimes needs to be adjusted as shown below.

There are two method available to adjust calculated spot position

### Adjust Ewald class
For temporary adjustments or relatively simple analysis it is recommended to set attributes: `beta`, `shift_x` and `shift_y`, `fine_scalling` in Ewald object directly.

 Those attributes are stored separately from the origin RHEED image.

 ### Adjust the RHEED image
If, for example, the `screen_scale` needs to be corrected this should be done directly on RHEED image object, but then the Ewald object has to be recrated. The same applies to the x/y shift of the RHEED image that was described in `Getting started` notebook.

In [ ]:
ew_si_111

In [ ]:
beta = 2.8
shift_y = -2.5
fine_scalling = 1.0

ew_si_111.beta = beta
ew_si_111.shift_y = shift_y
ew_si_111.fine_scalling = fine_scalling

ew_si_111.plot(auto_levels=1.0, marker="d", s=30, alpha=0.3, color="c")
plt.show()

### Adding the reconstruction
Having the (1x1) structure well adjusted we can add the (7x7) reconstruction.

In [ ]:
si_111_7x7 = Lattice.from_surface_hex(a=3.84*7)
si_111_7x7.rotate(alpha)
si_111_7x7

Create Ewald object for new lattice, and copy already adjusted attributes.

In [ ]:
ew_si_111_7x7 = Ewald(si_111_7x7, rheed_image)

ew_si_111_7x7.beta = beta
ew_si_111_7x7.shift_y = shift_y

ew_si_111_7x7.plot(auto_levels=1.0)
plt.show()

## Final image
Finally, prepare and save a plot with both reconstructions.

For clarity, here we use high pass filtered image.

In [ ]:
from xrheed.preparation.filters import high_pass_filter

sigma = 5.0 # in mm
sigma_px = sigma * rheed_image.ri.screen_scale
threshold = 0.8

hp_rheed_image = high_pass_filter(rheed_image, sigma=sigma, threshold=threshold)


In [ ]:
cm = 1 / 2.4

fig, ax = plt.subplots(figsize=(14 * cm, 10 * cm), constrained_layout=True)

hp_rheed_image.ri.plot_image(ax=ax, vmin=10, vmax=22, show_center_lines=False)

ew_si_111_7x7.plot(
    ax=ax, show_image=False, auto_levels=1.0, marker="|", s=10, alpha=0.7, color="y"
)

ew_si_111.plot(ax=ax, show_image=False, marker="d", s=40, alpha=0.7, color="c")

ax.set_xlabel(" ")
ax.set_ylabel(" ")
ax.set_xticks([])
ax.set_yticks([])
ax.set_ylim(-80, 0)
fig.set_dpi(100)
plt.show()
# fig.savefig()